# Addestramento con ppo

---

Setup ambiente e installazione librerie


In [0]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/


In [0]:
!nvidia-smi

Thu Jun 20 12:57:00 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install gym-retro

     |████████████████████████████████| 162.0MB 1.1MB/s 


In [0]:
!python -m retro.import /gdrive/My\ Drive/MachineLearning/StreetFighterII

Importing 59 potential games...
Importing StreetFighterIISpecialChampionEdition-Genesis
Imported 1 games


In [0]:
!cp /gdrive/My\ Drive/MachineLearning/StreetFighterII/state/* /usr/local/lib/python3.6/dist-packages/retro/data/stable/StreetFighterIISpecialChampionEdition-Genesis/

# Configurazione


---



In [0]:
import retro
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnLnLstmPolicy, CnnPolicy, CnnLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines.common.schedules import LinearSchedule
from stable_baselines import results_plotter
from stable_baselines import PPO2, A2C
import numpy as np
import gym


In [0]:
sts =  [#'RC-Mid',
        #'RZ-Mid',
        #'RD-Mid',
        #'Car',
        #'RB-Mid',
        #'RH-Mid',
        #'Bricks',
        #'RR-Mid',
        #'RG-Mid',
        #'RK-Mid',
        #'Barrel',
        #'RBal-Mid',
        #'RV-Mid',
        #'RS-Mid',
        #'RMb-Mid',
        #'RR-Easy',
        #'Champion.3S.Level1.RyuVsGuile',
        'GR-Easy',
        'GH-Mid']

gamename = 'StreetFighterIISpecialChampionEdition-Genesis'

In [0]:
n_cpu = 2

#env.close()
#Champion.3S.Level1.RyuVsGuile
env = SubprocVecEnv([lambda: retro.make('StreetFighterIISpecialChampionEdition-Genesis', state='CR-Easy' , scenario='scenario2',record='/gdrive/My Drive/MachineLearning/StreetFighterII/PPO')])


# Creazione modello e addestramento


---


L'agente viene addestrato per 8192 step su CnnPolicy e il reso dei parametri di default, come reward function viene usato il punteggio di gioco codificato nello scenario2

In [0]:
#PPO model and training single-stage
modelname = 'chunEasyStreetfighterIIPPO'
model = PPO2(CnnPolicy,env,n_steps=8192,verbose=1,tensorboard_log="/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/")

print('Start training for arcade')
env = SubprocVecEnv([lambda: retro.make('StreetFighterIISpecialChampionEdition-Genesis', state='CR-Easy', scenario='scenario2')])
#env = Monitor(env,'/gdrive/My Drive/MachineLearning/StreetFighterII/PPO')
model.set_env(env)
model.learn(total_timesteps=1000000)
model.save("/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/" + modelname)
env.close()
print("End training")
   

Nel caso di addestramenti successivi viene caricato un modello precedente

In [0]:
model = PPO2.load("/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/finalMidStreetfighterIIPPO.pkl", env,tensorboard_log="/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/")
#model.set_env(env)

Sezione con addestramento multi stage

In [0]:
# PPO model and training multi stage
modelname = 'streetfighterIIPPO'
model = PPO2(CnnPolicy,env,n_steps=8192,verbose=1,tensorboard_log="/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/")
#env.close()

for st in sts:
  print("Start training in state:")
  print(st)
  env = SubprocVecEnv([lambda: retro.make(gamename, state=st, scenario='scenario2')])
  model.set_env(env)
  model.learn(total_timesteps=1000000,tb_log_name=st)
  model.save("/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/" + st +"-"+ modelname)
  env.close()
  print("End training episode")
  


#model.learn(total_timesteps=1000000)
model.save("/gdrive/My Drive/MachineLearning/StreetFighterII/PPO/" + modelname)

# Test della rete


---

In  questa sezione vengono effettuati i test sul modello ottenuto e salvati i video dei risultati

[Risultato finale](https://drive.google.com/open?id=1-TuWhuhza6NlsUI0AYcI7TonqolrLF2k)

In [0]:
model.set_env(env)
obs = env.reset()
done = False
reward = 0

while not done:
  actions, _ = model.predict(obs)
  obs, rew, done, info = env.step(actions)
  #print(rew)
  reward += rew

  
print(reward)

[20.]


In [0]:
print(info)

({'enemy_health': 69, 'endgame-easy': 0, 'health': -1, 'endgame': 255, 'score': 336400, 'enemy_matches_won': 2, 'matches_won': 1, 'continuetimer': 0},)


In [0]:
#Plot del log
results_plotter.plot_results(["./log"], 10e6, 'timesteps', "Street Fighter II")

In [0]:
!tensorboard --logdir /gdrive/My\ Drive/MachineLearning/StreetFighterII/PPO/

In [0]:
!python /usr/local/lib/python3.6/dist-packages/retro/scripts/playback_movie.py /gdrive/My\ Drive/MachineLearning/StreetFighterII/PPO/StreetFighterIISpecialChampionEdition-Genesis-CR-Easy-000070.bk2

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena